# Estadisticas y Mantenimiento Flor Catorce

## Ajustes del NoteBook

In [26]:
# Dependencias
import requests
import random
from IPython.display import Markdown as md
import json
import pandas as pd
import os

In [27]:
# Servidores aptos para el sistema
centro = "26.61.16.123"
araucarias = "26.217.212.35"
cedis = "26.207.163.198"
pruebas = ""

In [28]:
datos ={
    "fecha": "2024-09-12",
    "efectivo_inicial": 24878.30,
    "servidor": centro
}

In [29]:
# Limite para dar mantenimiento a las ventas
corte = datos["efectivo_inicial"] / 2

In [30]:
# Productos para el mantenimiento
productos = ["034003", "042035"]
random.shuffle(productos) # Mezclar los productos

In [31]:
# Petición a la API de ventas
url_ventas = f"http://{datos['servidor']}:8000/mantenimiento/ventas"
api_ventas = requests.get(url_ventas, json=datos).json() # Convertir la respuesta a JSON (diccionario)

# Reportes de Folios

In [32]:
# Carpeta para almacenar los datos
os.mkdir(datos["fecha"])

In [33]:
# Reporte de las ventas
ventas = pd.DataFrame(api_ventas)
ventas.to_csv(f'./{datos["fecha"]}/Ventas_{datos["fecha"]}.csv', index=False)

ventas.loc[:,"efectivo"] = pd.to_numeric(ventas["efectivo"])
ventas.loc[:,"tarjeta"] = pd.to_numeric(ventas["tarjeta"])

In [34]:
# Reporte de las ventas facturadas (general)
ventas_facturadas = ventas[ventas["facturado"] == True]
ventas_facturadas.to_csv(f'./{datos["fecha"]}/Ventas_facturadas_{datos["fecha"]}.csv', index=False)

In [35]:
# Reporte de las ventas no facturadas (general)
ventas_no_facturadas = ventas[ventas["facturado"] == False]
ventas_no_facturadas.to_csv(f'./{datos["fecha"]}/Ventas_no_facturadas_{datos["fecha"]}.csv', index=False)

In [36]:
# Reporte ventas facturadas con efectivo
ventas_facturadas_efectivo = ventas_facturadas[(ventas_facturadas["efectivo"] > 0) & (ventas_facturadas["tarjeta"] == 0)]
ventas_facturadas_efectivo.to_csv(f'./{datos["fecha"]}/Ventas_facturadas_efectivo_{datos["fecha"]}.csv', index=False)

In [37]:
# Reporte ventas facturadas con tarjeta
ventas_facturadas_tarjeta = ventas_facturadas[(ventas_facturadas["tarjeta"] > 0) & (ventas_facturadas["efectivo"] == 0)]
ventas_facturadas_tarjeta.to_csv(f'./{datos["fecha"]}/Ventas_facturadas_tarjeta_{datos["fecha"]}.csv', index=False)

In [38]:
# Reporte ventas no facturadas con efectivo
ventas_no_facturadas_efectivo = ventas_no_facturadas[(ventas_no_facturadas["efectivo"] > 0) & (ventas_no_facturadas["tarjeta"] == 0)]
ventas_no_facturadas_efectivo.to_csv(f'./{datos["fecha"]}/Ventas_no_facturadas_efectivo_{datos["fecha"]}.csv', index=False)

In [39]:
# Reporte ventas no facturadas con tarjeta
ventas_no_facturadas_tarjeta = ventas_no_facturadas[(ventas_no_facturadas["tarjeta"] > 0) & (ventas_no_facturadas["efectivo"] == 0)]
ventas_no_facturadas_tarjeta.to_csv(f'./{datos["fecha"]}/Ventas_no_facturadas_tarjeta_{datos["fecha"]}.csv', index=False)

In [40]:
# Otras facturas (efeectivo y tarjeta)
otras_facturas = ventas[(ventas["efectivo"] > 0) & (ventas["tarjeta"] > 0)]
otras_facturas.to_csv(f'./{datos["fecha"]}/Otras_facturas_{datos["fecha"]}.csv', index=False)

In [41]:
# Ventas tarjeta y efectivo no facturadas
ventas_no_facturadas_mixtas = ventas_no_facturadas[(ventas_no_facturadas["efectivo"] > 0) & (ventas_no_facturadas["tarjeta"] > 0)]
ventas_no_facturadas_mixtas.to_csv(f'./{datos["fecha"]}/Otras_no_facturadas_{datos["fecha"]}.csv', index=False)

In [42]:
# Reporte datos encontrados
folios = ventas["folio"].unique()
folios = len(folios)

folios_facturados = ventas_facturadas["folio"].unique()
folios_facturados = len(folios_facturados)

folios_no_facturados = ventas_no_facturadas["folio"].unique()
folios_no_facturados = len(folios_no_facturados)

folios_facturados_efectivo = ventas_facturadas_efectivo["folio"].unique()
folios_facturados_efectivo = len(folios_facturados_efectivo)

folios_facturados_tarjeta = ventas_facturadas_tarjeta["folio"].unique()
folios_facturados_tarjeta = len(folios_facturados_tarjeta)

folios_no_facturados_efectivo = ventas_no_facturadas_efectivo["folio"].unique()
folios_no_facturados_efectivo = len(folios_no_facturados_efectivo)

folios_no_facturados_tarjeta = ventas_no_facturadas_tarjeta["folio"].unique()
folios_no_facturados_tarjeta = len(folios_no_facturados_tarjeta)

folios_facturados_mixtos = otras_facturas["folio"].unique()
folios_facturados_mixtos = len(folios_facturados_mixtos)

folios_no_facturados_mixtos = ventas_no_facturadas_mixtas["folio"].unique()
folios_no_facturados_mixtos = len(folios_no_facturados_mixtos)

In [43]:
# Diccionario con los datos filtrados
filtrado_ventas ={
    "folios": folios,
    "folios facturados": folios_facturados,
    "folios no facturados": folios_no_facturados,
    "folios facturados efectivo": folios_facturados_efectivo,
    "folios facturados tarjeta": folios_facturados_tarjeta,
    "folios no facturados efectivo": folios_no_facturados_efectivo,
    "folios no facturados tarjeta": folios_no_facturados_tarjeta,
    "Otros facturados": folios_facturados_mixtos,
    "Otros no facturados": folios_no_facturados_mixtos,
}

In [44]:
# Reporte de estadistica
estadistica = pd.DataFrame(filtrado_ventas, index=[0])
estadistica.to_csv(f'./{datos["fecha"]}/Estadistica_{datos["fecha"]}.csv', index=False)

# Mantenimiento de ventas

In [54]:
# Obtener los folios y sus totales mayores a 120 no facturados
df = mantenimiento_ventas = pd.read_csv(f'./{datos["fecha"]}/Ventas_no_facturadas_{datos["fecha"]}.csv')
df = df[["folio","total"]]

# Convertir a diccionario
dic_ventas = df.to_dict(orient="records")
